In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 100)

In [3]:
df = pd.read_csv('./data/aml_df.csv')

/home/ubuntu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (15,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [5]:
df.head()

,TARGET USI,Diagnostic ID,Gender,Race,Ethnicity,Age at Diagnosis in Days,First Event,Event Free Survival Time in Days,Vital Status,Overall Survival Time in Days,Year of Diagnosis,Year of Last Follow Up,Protocol,WBC at Diagnosis,Bone marrow leukemic blast percentage (%),Peripheral blasts (%),CNS disease,Chloroma,FAB Category,t(6;9),t(8;21),t(3;5)(q25;q34),t(6;11)(q27;q23),t(9;11)(p22;q23),t(10;11)(p11.2;q23),t(11:19)(q23:p13.1),inv(16),del5q,del7q,del9q,monosomy 5,monosomy 7,trisomy 8,trisomy 21,MLL,Minus Y,Minus X,Cytogenetic Code Other,Cytogenetic Complexity,Primary Cytogenetic Code,ISCN,FLT3/ITD positive?,FLT3/ITD allelic ratio,FLT3 PM,NPM mutation,CEBPA mutation,WT1 mutation,c-Kit Mutation Exon 8,c-Kit Mutation Exon 17,MRD at end of course 1,...,ENSG00000280913.1,ENSG00000280916.1,ENSG00000280927.1,ENSG00000280936.1,ENSG00000280953.1,ENSG00000280987.1,ENSG00000281005.1,ENSG00000281021.1,ENSG00000281026.1,ENSG00000281100.1,ENSG00000281103.1,ENSG00000281106.1,ENSG00000281128.1,ENSG00000281131.1,ENSG00000281183.1,ENSG00000281189.1,ENSG00000281195.1,ENSG00000281205.1,ENSG00000281207.1,ENSG00000281332.1,ENSG00000281344.1,ENSG00000281357.1,ENSG00000281358.1,ENSG00000281376.1,ENSG00000281383.1,ENSG00000281392.1,ENSG00000281398.1,ENSG00000281404.1,ENSG00000281406.1,ENSG00000281433.1,ENSG00000281445.1,ENSG00000281468.1,ENSG00000281490.1,ENSG00000281501.1,ENSG00000281527.1,ENSG00000281538.1,ENSG00000281555.1,ENSG00000281571.1,ENSG00000281628.1,ENSG00000281649.1,ENSG00000281691.1,ENSG00000281706.1,ENSG00000281741.1,ENSG00000281789.1,ENSG00000281896.1,ENSG00000281912.1,__no_feature,__ambiguous,__alignment_not_unique,Low Risk
0,TARGET-20-PADZCG,09A,Female,Unknown,Not Hispanic or Latino,5325,Relapse,314,Dead,536,1996,1998,CCG-2961,29.3,58,5.0,No,No,M4,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,,0,Normal,"46,XX[20]",No,NaN,No,No,No,No,Not done,Not done,Unknown,...,-2.422844,0.314777,1.745166,-6.927145,-6.121355,1.406833,0.995511,-0.223406,2.291398,-0.613341,3.901103,1.612242,-4.682102,-6.927145,0.884600,-2.471202,0.732776,0.678398,0.391151,1.822425,1.478602,1.379825,0.894175,-1.110556,-0.330849,0.091596,1.956429,-0.547052,-6.927145,-1.454984,1.123140,-2.930571,2.938798,1.768782,0.580973,-0.832714,-1.780950,0.132623,-0.682824,5.465605,-0.330849,-1.843229,-0.801464,-0.364707,2.164604,-0.152689,17.12435,14.60608,17.85268,0
1,TARGET-20-PAEAKL,09A,Female,Black or African American,Not Hispanic or Latino,5218,Relapse,134,Dead,163,1997,1997,CCG-2961,96.1,80,81.0,Yes,No,M5,No,No,No,No,Yes,No,Yes,No,No,No,No,No,No,No,No,Yes,No,No,,1,MLL,"46,XX,t(11:19)(q23:p13.1)[20]",No,NaN,No,No,No,No,Not done,Not done,Unknown,...,-5.804592,-6.927145,1.035256,-6.927145,-6.927145,0.113774,1.324330,-4.413837,0.664587,-6.927145,1.214346,-6.927145,-6.927145,-5.804592,-0.093968,-2.533837,-1.488071,-4.143354,-1.052135,0.929309,0.340124,0.951130,1.787532,-0.202203,6.320959,-0.544956,1.048812,0.496696,-6.927145,-2.799308,-0.914236,-1.944822,1.121176,3.167662,-2.617015,-1.892066,-3.124899,1.055543,-1.944822,3.838620,-2.177556,-0.717812,-0.888170,1.754821,0.456577,0.965496,16.79422,15.36352,19.67021,0
2,TARGET-20-PAEERJ,09A,Female,White,Not Hispanic or Latino,5082,Relapse,259,Dead,373,1997,1998,CCG-2961,71.9,60,41.0,No,No,M4,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,,1,Other,"46,XX,t(3:5)(q23:q33)[19]/46,XX[1]",Yes,0.55,No,No,No,Yes,Not done,Not done,Unknown,...,-3.822889,-3.009389,2.179869,-6.927145,-6.927145,1.949369,-1.003271,0.722174,3.025755,-3.822889,4.435130,-3.498090,-6.927145,-6.927145,1.320285,-4.509608,1.670870,-2.354343,-0.680935,1.476446,1.325430,0.512625,0.884345,-0.488087,-1.003271,-1.256426,2.912373,-0.209853,-6.927145,-1.351749,-0.068189,-2.815723,-0.081745,0.557041,0.309530,-2.566667,-1.858071,-0.470147,1.273137,6.077090,0.768241,-0.600677,-1.563662,-0.506252,2.328197,-2.909309,16.91605,14.56523,17.57099,0
3,TARGET-20-PAEFGR,09A,Female,White,Not Hispanic or Latino,549,Relapse,497,Alive,4022,1997,2008,CCG-2961,51.2,75,41.0,No,No,M2,

In [6]:
lasso_df = pd.read_csv('./data/lasso_features_data.csv')
lasso_df.drop(columns=['Unnamed: 0'], inplace=True)
lasso_columns = lasso_df.columns

In [7]:
# lasso_columns

In [8]:
final_val = df.sample(frac=0.1, random_state = 8)

final_X = final_val[lasso_columns].astype(float)
final_y = final_val.iloc[:, -1]

data = df.drop(index= final_val.index)
X = data[lasso_columns].astype(float)
y = data.iloc[:, -1]

In [9]:
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier
from xgboost import XGBClassifier
import random
from sklearn.model_selection import permutation_test_score
from IPython.display import display
from sklearn.model_selection import train_test_split, KFold

In [10]:
xgb = XGBClassifier(learning_rate = 0.01, max_depth = 7, n_estimators = 1000, random_state=8, n_jobs=-1)
rf = RandomForestClassifier(n_estimators=2000, max_depth=30, random_state=8, n_jobs=-1)

In [11]:
xgb = XGBClassifier(learning_rate = 0.01, max_depth = 3, n_estimators = 700, random_state=8, n_jobs=-1)
rf = RandomForestClassifier(n_estimators=1000, max_depth=20, random_state=8, n_jobs=-1)
kf = KFold(n_splits=5, shuffle=True)

rf_ll_performance = []
rf_acc_performance = []

xgb_ll_performance = []
xgb_acc_performance = []


for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    rf.fit(X_train, y_train)
    xgb.fit(X_train, y_train)

    p_rf = rf.predict_proba(X_test)
    p_xgb = xgb.predict_proba(X_test)
    p_rf_acc = rf.predict(X_test)
    p_xgb_acc = xgb.predict(X_test)

    rf_ll = log_loss(y_test, p_rf)
    xgb_ll = log_loss(y_test, p_xgb)
    rf_acc = accuracy_score(y_test, p_rf_acc)
    xgb_acc = accuracy_score(y_test, p_xgb_acc)
    
    rf_ll_performance.append(rf_ll)
    rf_acc_performance.append(rf_acc)
    xgb_ll_performance.append(xgb_ll)
    xgb_acc_performance.append(xgb_acc)

In [12]:
display(xgb_ll_performance)

display(rf_ll_performance)

[0.06391781192272901,
 0.2997434598952532,
 0.290205134563148,
 0.23863246268592775,
 0.22851841089626154]

[0.192422335743628,
 0.2919546618486977,
 0.26368649641195707,
 0.251422314251491,
 0.23640810455120195]

In [13]:
def log_loss_calc(model, X_train, y_train, X_test, y_test):
    """
    function that returns log loss value
    """
    model.fit(X_train, y_train)
    pred = model.predict_proba(X_test)
    ll = log_loss(y_test, pred)
    return ll

class XGBoostTuner():
    def __init__(self, param=None):
        self.param = param
        self.xgb_performance = {}
        self.best_parameters = []
    
    def xgb_kfolds(self, X, y, trees_list, depth, splits=5):
        """
        
        Parameters
        -----------
        X: data to train on
        y: labeled target data
        trees_list: list of numbers ex [2, 4, 6, 8, 10] that will decide number of trees
        depth: depth of tree

        Returns
        ----------------
        
        """
        kf = KFold(n_splits=splits, shuffle=True)

        models = []
        for i in trees_list:
            model = XGBClassifier(learning_rate=0.01, n_estimators= i * 100,
                                 depth = depth, n_jobs = -1, random_state=8)
            models.append(model)
        xgb1, xgb2, xgb3, xgb4, xgb5 = models

        scores = []
        for i in range(5):
            scores.append([])
        score1, score2, score3, score4, score5 = scores

        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            score1.append(log_loss_calc(xgb1, X_train, y_train, X_test, y_test))
            score2.append(log_loss_calc(xgb2, X_train, y_train, X_test, y_test))
            score3.append(log_loss_calc(xgb3, X_train, y_train, X_test, y_test))
            score4.append(log_loss_calc(xgb4, X_train, y_train, X_test, y_test))
            score5.append(log_loss_calc(xgb5, X_train, y_train, X_test, y_test))
        
        # self.xgb_performance = {}
        for i in range(len(trees_list)):
            self.xgb_performance[f'{trees_list[i]}00 trees'] = {'scores over k splits': scores[i], 'mean score': np.mean(scores[i])}
        return self.xgb_performance
    
    def best_params(self):
        mean_score = []
        for key in self.xgb_performance.keys():
            mean_score.append((self.xgb_performance[key]['mean score'], key))
        bp =  min(mean_score, key = lambda x: x[0])
        self.best_parameters.append(bp)
        return bp


In [14]:
xt1 = XGBoostTuner()
xgb_scores1 = xt1.xgb_kfolds(X, y, [5, 7, 10, 15, 20], depth=3)
print(xt1.best_params())

xt2 = XGBoostTuner()
xgb_scores2 = xt2.xgb_kfolds(X, y, [5, 7, 10, 15, 20], depth=5)
print(xt2.best_params())

xt3 = XGBoostTuner()
xgb_scores3 = xt3.xgb_kfolds(X, y, [5, 7, 10, 15, 20], depth=7)
print(xt3.best_params())

xt4 = XGBoostTuner()
xgb_scores4 = xt4.xgb_kfolds(X, y, [5, 7, 10, 15, 20], depth=10)
print(xt4.best_params())

xt5 = XGBoostTuner()
xgb_scores5 = xt5.xgb_kfolds(X, y, [5, 7, 10, 15, 20], depth=20)
print(xt5.best_params())

(0.2503426149782414, '500 trees')
(0.2423621205451588, '500 trees')
(0.22871608211969335, '500 trees')
(0.19402721256421257, '500 trees')
(0.2238916262357185, '500 trees')


In [ ]:
xgb = XGBClassifier(learning_rate = 0.01, max_depth = 10, n_estimators = 500, random_state=8, n_jobs=-1)
rf = RandomForestClassifier(n_estimators=1000, max_depth=20, random_state=8, n_jobs=-1)



In [27]:
rf.fit(X, y)
xgb.fit(X, y)

p_rf = rf.predict_proba(final_X)
p_xgb = xgb.predict_proba(final_X)
p_rf_acc = rf.predict(final_X)
p_xgb_acc = xgb.predict(final_X)

rf_ll = log_loss(final_y, p_rf)
xgb_ll = log_loss(final_y, p_xgb)
rf_acc = accuracy_score(final_y, p_rf_acc)
xgb_acc = accuracy_score(final_y, p_xgb_acc)

print(f'RF Performance: Accuracy: {rf_acc} Log Loss: {rf_ll}')
print('\n')
print(f'XGB Performance: Accuracy: {xgb_acc} Log Loss: {xgb_ll}')

RF Performance: Accuracy: 0.8571428571428571 Log Loss: 0.3129777319016821


XGB Performance: Accuracy: 0.8571428571428571 Log Loss: 0.3478189533177231
